In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Oppgave 1 - Forstå hvordan datasettene og transformermodellen er strukturert

#### 1.1

#### 1.3

Cross entropy er gitt ved $L(θ, D) = -\frac{1}{D \cdot n} \sum_{i=0}^{D-1} \sum_{j=0}^{n-1} \log \hat{Y}(i, k, j)$ hvor D er datapunktene, $\theta $
er parameterne, og $\hat{Y}$ er sannsynlighetsfordelingen. Det objektfunksjonen gjør er å sammenligne datapunktenen med onehot(y). Hvis $L(θ, D) = 0$ vil de være det samme, optimerte modellen og onehot(y) vil være identiske. Når dette inntreffer vil $\hat{y}$ være lik $y$.